In [1]:
import pandas, sqlalchemy
from datetime import datetime
import pytz

In [2]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

In [3]:
bnk = pytz.timezone('Asia/Bangkok')

In [4]:
Base = automap_base()

In [5]:
engine = create_engine('postgres+psycopg2://postgres@localhost:5436/anhperf_dev')

/Users/likit/.virtualenvs/anherf/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
Base.prepare(engine, reflect=True)

In [7]:
Base.classes.keys()

['alembic_version', 'roles', 'participants', 'registrations', 'checkins']

In [8]:
session = Session(engine)

In [9]:
Role = Base.classes['roles']

In [10]:
Participant = Base.classes['participants']

In [11]:
Registration = Base.classes['registrations']

In [12]:
speaker_role = Role(desc='speaker')
guest_role = Role(desc='guest')
vip_role = Role(desc='vip')
participant_role = Role(desc='participant')
invitee_role = Role(desc='invitee')

In [13]:
committee_role = Role(desc='committee')
session.add(committee_role)

In [14]:
session.add(Role(desc='rapporteur'))

In [15]:
session.add(speaker_role)
session.add(guest_role)
session.add(vip_role)
session.add(participant_role)
session.add(invitee_role)
session.commit()

In [103]:
bnk = pytz.timezone('Asia/Bangkok')

In [104]:
def add_new_participant(role, data):
    for item in data.iterrows():
        row = item[1]
        if not pandas.isna(row['user_email']):
            email = row['user_email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname)
                continue
        else:
            email = ''
        new_par = Participant(title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=email,
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'],
                        )
        new_par.roles = role # reflection automatically add s to the role attribute
        registered_at = None if pandas.isna(row['user_registered']) else row['user_registered']
        registered_at = bnk.localize(registered_at).astimezone(pytz.utc)
        pay_status = not pandas.isna(row['paid_at'])
        new_par.registrations_collection.append(Registration(
                                                    registered_at=registered_at,
                                                    pay_status=pay_status,
                                                    payment_required=True)
                                               )
        session.add(new_par)
    session.commit()

In [106]:
def add_new_committee(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if not pandas.isna(row['user_email']):
            email = row['user_email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=email,
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['user_registered']) else row['user_registered']
        if registered_at:
            if not isinstance(registered_at, datetime):  # some datetime is not a datetime object
                registered_at = datetime.strptime(registered_at, '%Y-%m-%d %H:%M:%S')
            registered_at = bnk.localize(registered_at).astimezone(pytz.utc)
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [95]:
def add_new_invitee(role, data):
    for idx,row in data.iterrows():
        year = 2018
        #regcode = '{}{:05}'.format(year, row['ID'])
        if not pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(old_id=None,
                        title=row['title'],
                         firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=email,
                         faculty=row['faculty'],
                         affil=row['affiliation'],
                         mobile=row['tel'],
                         delivery_address=row['address'],
                         position_type=row['position']
                        )
        new_par.roles = role
        registered_at = pytz.utc.localize(row['datetime']).astimezone(pytz.utc)
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [107]:
def add_new_rap(role, data):
    for idx,row in data.iterrows():
        year = 2018
        if not pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        #regcode = '{}{:05}'.format(year, row['ID'])
        new_par = Participant(old_id=None,
                        title=row['title'],
                         firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=email,
                         affil=row['university'],
                         mobile=row['mobile'],
                         position_type=row['position']
                        )
        new_par.roles = role
        registered_at = bnk.localize(row['registered']).astimezone(pytz.utc)
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [108]:
def add_new_speakers(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if pandas.isna(row['user_email']):
            email = row['user_email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=row['user_email'].strip(),
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['user_registered']) else row['user_registered']
        if registered_at:
            if not isinstance(registered_at, datetime):  # some datetime is not a datetime object
                registered_at = datetime.strptime(registered_at, '%Y-%m-%d %H:%M:%S')
            registered_at = bnk.localize(registered_at).astimezone(pytz.utc)
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [109]:
def add_new_speakers_2(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=row['email'].strip(),
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         position_type=row['position'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['registered']) else row['registered']
        if registered_at:
            if not isinstance(registered_at, datetime):  # some datetime is not a datetime object
                registered_at = datetime.strptime(registered_at, '%Y-%m-%d %H:%M:%S')
            registered_at = bnk.localize(registered_at).astimezone(pytz.utc)
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [110]:
def add_new_committee_network(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if not pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        
        new_par = Participant(title=row['title'],
                         firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=email,
                         affil=row['affiliation'],
                         mobile=row['mobile'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['registered']) else row['registered']
        if registered_at:
            if not isinstance(registered_at, datetime):  # some datetime is not a datetime object
                registered_at = datetime.strptime(registered_at, '%Y-%m-%d %H:%M:%S')
            registered_at = bnk.localize(registered_at).astimezone(pytz.utc)
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [105]:
participant_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='participants')
participant_role = session.query(Role).filter(Role.desc=='participant').first()
add_new_participant(participant_role, participant_data)

วิวรรธน์


In [111]:
rap_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='rapporteurs')
rapporteur_role = session.query(Role).filter(Role.desc=='rapporteur').first()
add_new_rap(rapporteur_role, rap_data)

อภิรดี เจริญนุกูล


In [112]:
committee_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='committee')
committee_role = session.query(Role).filter(Role.desc=='committee').first()
add_new_committee(committee_role, committee_data)

ศุกร์ใจ เจริญสุช
ธนชัย พนาพุฒิ
วัชราภรณ์ เชี่ยววัฒนา
แสงเทียน ธรรมลิขิตกุล
พีรนันท์ วิศาลสกุลวงษ์
ภาวิดา พุทธิขันธ์
บุหงา ตโนภาส
รัตนา จารุวรรโณ
อังคณา สุเมธสิทธิกุล


In [114]:
invitee_data = pandas.read_excel('invitee_04112018.xlsx')
invitee_role = session.query(Role).filter(Role.desc=='invitee').first()
add_new_invitee(invitee_role, invitee_data)

รุ่งเพ็ชร สกุลบำรุงศิลป์
วนิดา นพพรพันธุ์
ธานี กล่อมใจ
ขวัญตา บุญวาศ
ศิริเกษม ศิริลักษณ์
ธนันต์นิภา   ภัคศุขนิธิวัฒน์
ดร.ศรีสุดา วงศ์วิเศษกุล
อติญาณ์ ศรเกษตริน
สุกิจ  พันธุ์พิมานมาศ
วรุณยุพา รอยกุลเจริญ
อรทิพา   ส่องศิริ
นวลใย พิศชาติ
เสาวมาศ คุณล้าน  เถื่อนนาดี
รัชนี สรรเสริญ
สุชาดา นิ้มวัฒนากุล
สมใจ ลือวิเศษไพบูลย์
วิภา เพ็งเสงี่ยม
วรรณี ตปนียากร
ปานเทพ รัตนากร
ธวัชชัย ศักดิ์
มัณฑนา เหมชะญาติ
ยุวดี รอดจากภัย
ณิชารีย์ ใจคำวัง
ประยุทธ ภูวรัตนาวิวิธ
นิมัศตูรา แว
อุทัยวรรณ กาญจนกามล
กฤช จารุชาต
วารี กังใจ
พิมพ์เพชร สุขุมาลไพบูลย์
ประไพพิศ สิงหเสม
ชนัตถา พลอยเลื่อมแสง
ปัทมา ผ่องศิริ
ธนะเศรษฐ์ ง้าวหิรัญพัฒน์
พิพัฒน์  แก้วอุดม
พรเจริญ บัวพุ่ม
จิราพร วัฒนศรีสิน
ชลาลัย หาญเจนลักษณ์
นงเยาว์ สมพิทยานุรักษ์
วรรณภา ประทุมโทน
วิภาวรรณ  ชะอุ่ม เพ็ญสุขสันต์
นงเยาว์ สมพิทยานุรักษ์
การุณ เวโรจน์
นิภารัตน์ จันทร์แสงรัตน์
รุ่งโรจน์ ธนาวงษ์นุเวช
ศันสณี รัชชกูล
สุโรจน์ ศุภเวคิน
อมร ลีลารัศมี
นภา วงษ์ศรี
สราพร มัทยาท
สุทธิพงศ์ วัชรสินธุ
สุวิมล ล้วนเส้ง
นภาดล สีหพันธุ์
วิชิต โนสูงเนิน
ประยุทธ ภูวรัตนาวิวิธ


In [115]:
speakers_data = pandas.read_excel('register 06.11.2018.xlsx', 'speakers')
speaker_role = session.query(Role).filter(Role.desc=='speaker').first()
add_new_speakers(speaker_role, speakers_data)

In [116]:
speakers_data_2 = pandas.read_excel('register 06.11.2018.xlsx', 'Sheet1')
add_new_speakers_2(speaker_role, speakers_data_2)

In [117]:
network_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='network')
committee_role = session.query(Role).filter(Role.desc=='committee').first()
add_new_committee_network(committee_role, network_data)

siriwat Tiptaradol


In [118]:
notitles = []
for participant in session.query(Participant):
    if participant.title == 'NaN':
        notitles.append(participant)

In [119]:
for p in notitles:
    p.title='คุณ'
    print(p.title, p.firstname, p.lastname)

คุณ ศิริตรี สุทธจิตต์
คุณ วัชราภรณ์ เชี่ยววัฒนา
คุณ กฤษณา  ศิริวิบุลยกิติ
คุณ พัชริี  ดำรงสุนทรชัย
คุณ จารุภา จิรโสภณ


In [79]:
for p in notitles:
    session.add(p)
session.commit()

In [80]:
p = session.query(Participant).filter(Participant.id==546).first()
email1, email2 = p.email.split(', \n')
p.email = email2
session.add(p)
session.commit()

ValueError: not enough values to unpack (expected 2, got 1)

In [81]:
p = session.query(Participant).filter(Participant.lastname=='ดำรงสุนทรชัย').first()

In [82]:
p.firstname = 'พัชรี'
session.add(p)
session.commit()

In [83]:
p.firstname

'พัชรี'

In [84]:
p.email = 'bow_bow999@hotmail.com'
session.add(p)
session.commit()

In [88]:
pytz.utc.localize(datetime.now()).astimezone(pytz.utc)

datetime.datetime(2018, 11, 11, 20, 13, 37, 16352, tzinfo=<UTC>)